In [10]:
import networkx as nx
from collections import deque
from collections import namedtuple
Choice = namedtuple("Choice","task place")
Problemdata = namedtuple("ProblemData", "routing precedences costs")

def fix_weights(G):
    for _,_,d in G.edges(data=True):
        if 'weight' in d:
            d['weight'] = float(d['weight'].replace("\"", ""))

## Loading

In [11]:
GPos = nx.Graph(nx.nx_pydot.read_dot("prerequisites/Precedences.dot"))
fix_weights(GPos)
GPred = nx.DiGraph(nx.nx_pydot.read_dot("prerequisites/Precedences.dot"))
GTaskCost = nx.Graph(nx.nx_pydot.read_dot("prerequisites/Costs.dot"))
fix_weights(GTaskCost)
problemdata = Problemdata(GPos, GPred, GTaskCost)

## Node-Class

In [12]:
class Node:
    def __init__(self, problemdata) -> None:
        self.data = problemdata
        self.choices = []
        self.OpenTasks = {x for x in problemdata.precedences}
        self.quality = 0

    def create_Child(self, choice) -> "Node":
        """Creates a new Node objecte by copying self and applying the choice"""
        new_node = Node(self.data)
        new_node.choices = self.choices.copy()
        new_node.choices.append(choice)
        new_node.OpenTasks = self.OpenTasks.copy()
        new_node.OpenTasks.remove(choice.task)
        new_node.quality = self.quality + self.data.task_quality[choice.task]
        return new_node


    def expand(self):
        """Create all child nodes by applying all possible 
        choices (combinations of tasks and places)"""
        children = []
        for task in self.OpenTasks:
            if self.all_precedences_fulfilled(task):
                for place in self.possible_locations:
                    choice = Choice(task, place)  # Assuming Choice is a defined class
                    child = self.create_Child(choice)
                    children.append(child)
        return children

    @property
    def is_finished(self) -> bool:
        return len(self.OpenTasks) == 0

    def all_precedences_fulfilled(self, task) -> bool:
        return not any(pr in self.OpenTasks for pr in self.data.precedences[task])

    @property
    def possible_locations(self) -> list[str]:
        if len(self.choices) != 0:
            return self.data.routing[self.choices[-1].place]
        return self.data.routing

## Branch And Bound 

In [13]:
# you can use pop, popleft, extend, append, and appendleft with the deque object
#https://docs.python.org/3/library/collections.html#collections.deque

def branch_and_bound(problem_data):
    start = Node(problem_data)
    queue = deque([start])
    best = None
    visited_nodes = 0
    max_queue_size = 1
    while len(queue) > 0:
        current_node = queue.popleft()
        visited_nodes += 1

        if current_node.is_finished:
            if best is None or current_node.quality > best.quality:
                best = current_node
            continue

        for child in current_node.expand():
            if best is None or child.quality > best.quality:  # Simple bounding condition
                queue.append(child)

        max_queue_size = max(max_queue_size, len(queue))
    return best, visited_nodes, max_queue_size

## Experiments

In [15]:
def test_create_child(self):
    node = Node(problemdata)
    choice = Choice(task="Fressen", place="Bad")
    child = node.create_Child(choice)
    assert choice in child.choices
    assert choice.task == child.OpenTasks
